In [16]:
import os
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

In [2]:
SRC = '/Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input'
videos = [file for file in os.listdir(SRC) if file.endswith("MP4") and file.startswith("GX")]

In [4]:
video = videos[-2]

In [6]:
video

In [7]:
csvs = [file for file in os.listdir(SRC) if file.endswith("intervals-errfilter.csv") and file.startswith(video.removesuffix('.MP4'))]

In [8]:
csvs

In [9]:
csv = csvs[1]

In [10]:
csv

In [11]:
df = pd.read_csv(os.path.join(SRC, csv))

In [86]:
df

In [91]:
vidcap = cv2.VideoCapture(os.path.join(SRC, video))
image = np.zeros((28196+21, 1424+2160), dtype=np.float32)
gaussian_kernel  = cv2.getGaussianKernel(ksize=21, sigma=5)

In [38]:
gaussian_kernel[:, :, np.newaxis].shape

In [25]:
vidcap.get(cv2.CAP_PROP_FRAME_WIDTH), vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [ ]:
pbar = tqdm(total=vidcap.get(cv2.CAP_PROP_FRAME_COUNT), desc=f"Printing image from video")
for index, (frame_n, x_shift, y_shift) in df.iterrows():
    ret, frame = vidcap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cropped_frame = frame[1800:1821, :]
    cropped_frame = cropped_frame.astype(np.float32)
    weighted_frame = cropped_frame * gaussian_kernel
    
    # Add the weighted frame to the image at the specified position
    x_start = int(-x_shift)
    y_start = int(-y_shift)

    # Ensure the region is within image boundaries
    x_end = x_start + weighted_frame.shape[1]
    y_end = y_start + weighted_frame.shape[0]

    print(y_start, y_end, x_start, x_end)
    image[y_start:y_end, x_start:x_end] += weighted_frame
    
    pbar.update(1)
    

In [97]:
image.shape[0]/10, image.shape[1]/10

In [98]:
cv2.normalize(image, image, 0, 255, cv2.NORM_MINMAX)

image = cv2.resize(image, (int(image.shape[0]/10), int(image.shape[1]/10)))
cv2.imshow('Cropped Frame', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
pbar.close()

In [95]:
cv2.imwrite('test.png', image)

In [66]:
weighted_frame

In [71]:
frame

In [41]:
cropped_frame * gaussian_kernel[:, :, np.newaxis]

In [57]:
np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]])

In [59]:
np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]]) * cv2.getGaussianKernel(3, 1) 

In [53]:
cv2.getGaussianKernel(3, 1) 